In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
import json
import sys
import re
import logging
import os
from pathlib import Path
from functools import partial
from itertools import chain
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
# from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from gioutils.utils import parse_offset, today, get_next_n
from api.data.base import data_path, radar_path
from gioutils.gui.base import View, store
from api.gui.params import curves, bbg_params, spot_params, fwd_params
from gioutils.blpw import BlpQuery
from gioutils.utils import RepeatedTimer
from gioutils.ezutils import bh, bhs
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio

pd.options.plotting.backend = "plotly"

In [3]:
from api.gui.collectors import DataCollector, SpotForward, Futures, Market, Mtgs, FwdMatrix
from api.gui.radar2 import EditableGrid
from api.gui.tables import SpotTable, FutureTable, GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.grids import SpotGrid, FutureGrid, MtgGrid, MatGrid, GroupMktGrid # GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable

In [4]:
bq = BlpQuery(timeout=50000).start()

In [5]:
ccys = [
    'EUR', 
    'USD', 
    'GBP'
]

In [6]:
curves_df = pd.DataFrame(curves).set_index('ccy').loc[ccys]

In [8]:
### Fly Grid

In [15]:
fly_grid = EditableGrid()
# fly_grid.file_tf.v_model = 'test2'
# fly_grid.load_btn.fire_event('click', None)

usd_spot = SpotGrid(gen='USD SOFR')
usd_fut = FutureGrid(gen='USD SOFR', ccy='USD')
usd_chart = KeyChart(gen='USD SOFR', chart_kwargs=dict(height=300))

eur_spot = SpotGrid(gen='EURIBOR A 6M*', title='EURIBOR')
eur_spot2 = SpotGrid(gen='EUR EONIA')
eur_fut = FutureGrid(gen='EURIBOR A 6M*', title='EURIBOR', ccy='EUR')
eur_chart = KeyChart(gen='EURIBOR A 6M*', chart_kwargs=dict(height=300))

gbp_spot = SpotGrid(gen='GBP SONIA')
gbp_fut = FutureGrid(gen='GBP SONIA', ccy='GBP')
gbp_chart = KeyChart(gen='GBP SONIA', chart_kwargs=dict(height=300))

mkt_equity = GroupMktGrid(group='equity', title='EQUITY')
mkt_fx = GroupMktGrid(group='fx', pct=False, title='FX')
mkt_comdty = GroupMktGrid(group='commodity', title='COMDTY')

mtg_usd = MtgGrid(ccy='USD', title='USD', multipliers=100)
mtg_eur = MtgGrid(ccy='EUR', title='EUR', multipliers=100)
mtg_gbp = MtgGrid(ccy='GBP', title='GBP', multipliers=100)

mat_usd_live = MatGrid(ccy='USD', curve_type='live', title='Fwd live', multipliers=100)
mat_eur_live = MatGrid(ccy='EUR', curve_type='live', title='Fwd live', multipliers=100)
mat_gbp_live = MatGrid(ccy='GBP', curve_type='live', title='Fwd live', multipliers=100)

mat_usd_chg = MatGrid(ccy='USD', curve_type='change', title='Fwd chg', multipliers=10000, red_negative=True)
mat_eur_chg = MatGrid(ccy='EUR', curve_type='change', title='Fwd chg', multipliers=10000, red_negative=True)
mat_gbp_chg = MatGrid(ccy='GBP', curve_type='change', title='Fwd chg', multipliers=10000, red_negative=True)

In [16]:
### Govt Bond and Fut

In [17]:
govt_bond_sep_map = [
     'US30Y',
     'UK30Y',
     'GER30Y',
     'FR30Y',
     'ITA30Y',
     'POR30Y',
     # 'SPA30Y'
]

govt_bond_fut_sep_map = [
     'USA',
     'G A',
     'UBA',
     'OATA',
]

In [18]:
mkt_govt_bond = GroupMktTalbe(
    group='govt bonds', 
    pct=False, 
    precision=3,
    change_precision=2,
    styler_kwargs=dict(
        multipliers={'change': 100},
        finalize=custom_styler_row_line,
        finalize_kwargs=dict(mapping=govt_bond_sep_map)
    ),
    field='yield',   
)
mkt_govt_bond_fut = CustomGovtBondFutTable(
    styler_kwargs=dict(
        finalize=custom_styler_row_line,
        finalize_kwargs=dict(mapping=govt_bond_fut_sep_map)
    )
)

In [19]:
### Monitor

In [20]:
class Monitor(View):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def make_widgets(self, **kwargs):
        pass
    
    def make_view(self):
        self.container = v.Container(class_="ma-0 pa-0", style_=f"background: {bg_color}")
        self.view = self.container

#     @property
#     def refresher(self):
#         return self._refresher
    
#     @refresher.setter
#     def refresher(self, value):
#         self.refresh_view2()
        
    def render(self, **kwargs):
        dc = kwargs.get('dc')
        # self.refresher = True
        
    # def refresh_view2(self):
        # dc = self.dc
        children = [
            v.Row(
                children=[
                    v.Col(
                        cols=3, 
                        children=[
                            w.VBox([
                                w.HBox([
                                    usd_fut.get_widget(dc), mtg_usd.get_widget(dc)
                                ]), 
                                 w.HBox([
                                     usd_spot.get_widget(dc),  
                                     w.VBox([
                                         mkt_equity.get_widget(dc),  
                                         mkt_fx.get_widget(dc), 
                                         mkt_comdty.get_widget(dc)
                                     ])
                                ])  
                            ])

                        ]
                    ),
                    v.Col(
                        cols=3, 
                        children=[
                            w.VBox([                                
                                w.HBox([
                                    eur_fut.get_widget(dc), 
                                    mtg_eur.get_widget(dc)]), 
                                w.HBox([eur_spot.get_widget(dc), 
                                        mkt_govt_bond.get_widget(dc), 
                                        mkt_govt_bond_fut.get_widget(dc)]),
                                # eur_spot2.get_widget(dc)
                            ])
                        ]
                    ),

                    v.Col(
                        cols=3, 
                        children=[
                            w.VBox([                                
                                w.HBox([gbp_fut.get_widget(dc), mtg_gbp.get_widget(dc)]),
                                gbp_spot.get_widget(dc),                                
                            ])

                        ]
                    ),
                    v.Col(
                        cols=3, 
                        children=[
                          

                        ]
                    ),
                ]
            ),            
            v.Row(
                children=[
                    v.Col(cols=4, children=[usd_chart.get_widget(dc)]),
                    v.Col(cols=4, children=[eur_chart.get_widget(dc)]),
                    v.Col(cols=4, children=[gbp_chart.get_widget(dc)]),
                ]
            ),
            # v.Row(children=[v.Col(cols=5, children=[fly_grid.expr_dg.view])]),
            v.Row(
                children=[
                    v.Col(cols=4, children=[mat_usd_live.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_eur_live.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_gbp_live.get_widget(dc)]),
                ]
            ),
            v.Row(
                children=[
                    v.Col(cols=4, children=[mat_usd_chg.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_eur_chg.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_gbp_chg.get_widget(dc)]),
                ]
            )
        ]
        self.container.children = children
        
        
    def link(self, **kwargs):
        super().link(**kwargs)

In [21]:
class Monitor(View):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def make_widgets(self, **kwargs):
        pass
    
    def make_view(self):
        self.container = v.Container(class_="ma-0 pa-0", style_=f"background: {bg_color}")
        self.view = self.container

#     @property
#     def refresher(self):
#         return self._refresher
    
#     @refresher.setter
#     def refresher(self, value):
#         self.refresh_view2()
        
    def render(self, **kwargs):
        dc = kwargs.get('dc')
        # self.refresher = True
        
    # def refresh_view2(self):
        # dc = self.dc
        children = [
            v.Row(
                children=[
                    v.Col(
                        cols=5, 
                        children=[
                            w.VBox([
                                w.HBox([
                                    mtg_usd.get_widget(dc),
                                    mtg_eur.get_widget(dc),
                                    mtg_gbp.get_widget(dc)
                                ]), 
                                w.HBox([
                                    usd_fut.get_widget(dc), 
                                    eur_fut.get_widget(dc), 
                                    gbp_fut.get_widget(dc), 
                                ]), 
                                 w.HBox([
                                     usd_spot.get_widget(dc),  
                                     eur_spot.get_widget(dc),
                                     gbp_spot.get_widget(dc)
                                ])  
                            ])
                        ]
                    ),
                    v.Col(
                        cols=2, 
                        children=[
                            w.VBox([
                                mkt_equity.get_widget(dc),
                                mkt_fx.get_widget(dc),
                                mkt_comdty.get_widget(dc),                                
                            ])
                        ]
                    ),
                ],
            ),  
            v.Row(
                children=[
                    v.Col(cols=4, children=[usd_chart.get_widget(dc)]),
                    v.Col(cols=4, children=[eur_chart.get_widget(dc)]),
                    v.Col(cols=4, children=[gbp_chart.get_widget(dc)]),
                ]
            ),
            # v.Row(children=[v.Col(cols=5, children=[fly_grid.expr_dg.view])]),
            v.Row(
                children=[
                    v.Col(cols=4, children=[mat_usd_live.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_eur_live.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_gbp_live.get_widget(dc)]),
                ]
            ),
            v.Row(
                children=[
                    v.Col(cols=4, children=[mat_usd_chg.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_eur_chg.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_gbp_chg.get_widget(dc)]),
                ]
            )
        ]
        self.container.children = children
        
        
    def link(self, **kwargs):
        super().link(**kwargs)

In [22]:
m = Monitor()

In [23]:
### Collector

In [24]:
dc = DataCollector(
    curves=curves_df.melt()['value'].drop_duplicates().dropna().to_list(),
    bq=bq,
    collectors={
        'eg': fly_grid,
        'sf': SpotForward(spot_params=spot_params, fwd_params=fwd_params),
        'fut': Futures(),
        'mkt': Market(),
        'mtgs': Mtgs(),
        'mat': FwdMatrix()
    },
    renderers={
        'monitor': m,
    }
)

In [25]:
gc = GlobalConfig(dc=dc)

In [26]:
class Radar(View):

    def __init__(self, curves=None, **kwargs):
        self._curves = curves
        super().__init__(**kwargs)

    def make_widgets(self):
        editable_grid = EditableGrid(curves=self._curves)
        tabs_bars = v.Tabs(
            # dark=True,
            v_model=None,
            children=[
                v.Tab(children=['Radar']),
                v.Tab(children=['Flies']),
                v.Tab(children=['Wathclist'])
            ]
        )

        t1 = v.Container(
            children=[
                v.Row(children=[v.Col(children=[gc.view])]),
                v.Row(children=[v.Col(children=[m.view])])
            ],
            class_='mx-0',
        )
        t2 = v.Container(
            children=[
                v.Row(children=[v.Col(children=[fly_grid.param_box])]),
                v.Row(children=[v.Col(children=[fly_grid.dg])])
            ],
            class_='mx-0',
        )
        
        t3 = v.Container(
            children=[
                v.Row(children=[v.Col(children=[fly_grid.expr_dg.view])]),
            ],
            class_='mx-0',
        )

        tabs_items = v.TabsItems(
            v_model=None,
            # dark=True,
            children=[
                v.TabItem(children=[t1]),
                v.TabItem(children=[t2]),
                v.TabItem(children=[t3])
            ]
        )
        
        self.tabs_bars = tabs_bars
        self.t1 = t1
        self.t2 = t2
        self.t3 = t3
        self.tabs_items = tabs_items

    def make_view(self):
        self.view = v.Card(
            children=[
                self.tabs_bars,
                self.tabs_items
            ]
        )

    def link(self):
        w.jslink((self.tabs_bars, 'v_model'), (self.tabs_items, 'v_model'))
        super().link()

In [ ]:
Radar()

In [ ]:
# from ipyflex import FlexLayout
# widgets = { 'Widget 1': w.HTML('<h1> Widget 1</h1>'),
#             'Widget 2': w.HTML('<h1> Widget 2</h1>'),
#             'Widget 3': w.HTML('<h1> Widget 3</h1>'),
#             'Widget 4': w.HTML('<h1> Widget 4</h1>')
#         }
# dashboard = FlexLayout(
#     widgets,
#     header=True,
#     layout_config = {'borderLeft': False, 'borderRight': False, 'enableSection': True},
# )
# dashboard